# Przetwarzanie Grafiki i Muzyki - laboratorium nr 4

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Zadanie 1 (1.5 punkta)
Proszę zaimplementować (ręcznie) konwersję z formatu RGB na YCbCr (https://en.wikipedia.org/wiki/YCbCr) i przetestować na wybranym przez siebie obrazku. Wzór:

$$
\begin{align}
Y &= 0.299 \cdot R + 0.587\cdot G + 0.114\cdot B \\
C_B &= 0.5 -0.169\cdot R - 0.331 \cdot G + 0.5\cdot B \\
C_R &= 0.5 + 0.5\cdot R - 0.419\cdot G - 0.081\cdot B,
\end{align}
$$

gdzie $R, G, B \in [0,1]$.

In [73]:
import os

import cv2
import numpy as np


def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.169, -.331, .5], [.5, -.419, -.081]])
    ycbcr = im.dot(xform.T)
    ycbcr[:, :, [1, 2]] += 0.5
    return np.float32(ycbcr)


image1 = cv2.imread('./img/LENA_512.jpg')
img_scaled = image1.astype(np.float32) / 255.0
ycbcr_result = rgb2ycbcr(img_scaled)
image1 = cv2.imread('./img/LENA_512.jpg')
# użycie gotowej funkcji dla porównania
imgYCC = cv2.cvtColor(image1, cv2.COLOR_BGR2YCR_CB)
cv2.imshow("ycbcr", ycbcr_result)
cv2.imshow("ycbcr_expected", imgYCC)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Zadanie 2 (1.5 punkta)

Proszę zaimplementować ręcznie konwersję z formatu YCbCr na RGB (https://en.wikipedia.org/wiki/YCbCr). Przetestuj kod na obrazie wynikowym z poprzedniego zadania. Wzór:

$$
\begin{align}
    R &= Y + 1.402*(C_R - 0.5) \\
    G &= Y - 0.344*(C_B - 0.5)-0.714*(C_R - 0.5) \\
    B &= Y + 1.772*(C_B - 0.5)
\end{align}
$$

gdzie $Y, C_R, C_B \in [0, 1]$.

In [71]:
import os
import cv2
import numpy as np

def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.344, -0.714], [1, 1.772, 0]])
    im[:, :, [1, 2]] -= 0.5
    return np.uint8(im.dot(xform.T) * 255.0)

rgb_result = ycbcr2rgb(ycbcr_result)
cv2.imshow("rgb", rgb_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Zadanie 3 (1 punkt)
Proszę zaimplementować konwersję z formatu RGB na HLS i wyświetlić składowe H, L oraz S. Tutaj można użyć gotowej metody z pakietu OpenCV (cv2.COLOR_BGR2HLS).

In [11]:
img = cv2.imread('./img/LENA_512.jpg')
img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
h, l, s = cv2.split(img)
cv2.imshow("h", h)
cv2.imshow("l", l)
cv2.imshow("s", s)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Zadanie 4 (2 punkty)

Proszę stworzyć obrazek zawierający tylko zielone papryczki (analogicznie do przykładu z wykładu). Użyj obrazku "peppers.png".

HINT - zielone papryczki znajdują się tam, gdzie $R\in [20, 120]$ i $G \in [20, 160]$. R i G to natężenie składowych koloru w modelu RGB.

In [52]:
import cv2
import numpy as np

image = cv2.imread("./img/peppers.png", cv2.IMREAD_COLOR)
height, width, channels = image.shape
for i in range(0, height):
    for j in range(0, width):
        b, g, r = image[i][j]
        pixel = image[i][j]
        if not ((r >= 20 and r <= 120) and (g >= 20 and g <= 160)):
           image[i][j] = 0

cv2.imshow("Green Peppers", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Zadanie 5 (2 punkty)
Proszę dokonać detekcji skóry na zdjęciu "face.jpg", używając modelu HLS.

HINT - skóra znajduje się tam, gdzie spełniona jest następująca reguła:
$$
\left(S \geq 50\right) \land \left(0.5 < \frac{L}{S} < 3 \right) \land \left( \left(H \leq 14\right) \lor \left(H \geq 165\right) \right) 
$$

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/face.jpg)

In [69]:
image = cv2.imread("./img/face.jpg")
height, width = image.shape[:2]
hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
h, l, s = cv2.split(hls)
for i in range(height):
    for j in range(width):
        quotient = 0 if s[i][j] == 0 else l[i][j] / s[i][j]
        if not (s[i][j] >= 50 and quotient > 0.5 and quotient < 3.0 and (h[i][j] <= 14 or h[i][j] >= 165)):
            image[i][j] = 0

cv2.imshow("face", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

##  Zadanie 6 (2 punkty)
Proszę dokonać detekcję efektu czerwonych oczu na zdjęciu "face1.jpg".

HINT - czerwone oczy znajdują się tam, gdzie jest spełniona następująca reguła:
$$
\left( L \geq 64 \right) \land \left(S \geq 100\right) \land \left(0.5 < \frac{L}{S} < 1.5 \right) \land \left( \left(H \leq 7\right) \lor \left(H \geq 162\right) \right) 
$$

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/face1.jpg)

In [68]:
image = cv2.imread("./img/face1.jpg")
height, width = image.shape[:2]
hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
h, l, s = cv2.split(hls)
for i in range(height):
    for j in range(width):
        quotient = 0 if s[i][j] == 0 else l[i][j] / s[i][j]
        if not (l[i][j] >= 64 and s[i][j] >= 100 and quotient > 0.5 and quotient < 1.5 and (h[i][j] <= 7 or h[i][j] >= 162)):
            image[i][j] = 0

cv2.imshow("face", image)
cv2.waitKey(0)
cv2.destroyAllWindows()